In [ ]:
# !pip install torch==2.2.0+cu118 torchtext -f https://download.pytorch.org/whl/torch_stable.html

In [ ]:
import torch
import torchtext
import torch.nn as nn
import torch.nn.functional as F

import random
import math
import time

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [ ]:
SEED = 1234
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

### Dataset

This task is to translate English to German

In [ ]:
from datasets import load_dataset

dataset = load_dataset("kvush/english_thai_texts")

In [ ]:
dataset

In [ ]:
print(dataset["train"])

In [ ]:
# train_data, valid_data, test_data = (dataset["train"], dataset["validation"], dataset["test"])

In [ ]:
new_dataset = dataset.remove_columns(['input_ids', 'translated_ids', '__index_level_0__'])

In [ ]:
new_dataset

In [ ]:
train_data = new_dataset["train"]
train_data

In [ ]:
print(train_data[0])
print(train_data[1])
print(train_data[2])
print(train_data[3])
print(train_data[4])

### Tokenizer

In [ ]:
from pythainlp.tokenize import word_tokenize

text = "บริการหอยเชลล์ทั้งหมด - โดดเด่นในทุกด้าน!"

In [ ]:
th_nlp = word_tokenize
th_nlp(text, engine="newmm")

['บริการ',
 'หอย',
 'เชลล์',
 'ทั้งหมด',
 ' ',
 '-',
 ' ',
 'โดดเด่น',
 'ใน',
 'ทุก',
 'ด้าน',
 '!']

In [ ]:
import spacy
en_nlp = spacy.load("en_core_web_sm")
en_nlp

In [ ]:
string = "Service, scallops, all - top notch in every way!"
[token.text for token in en_nlp.tokenizer(string)]

['Service',
 ',',
 'scallops',
 ',',
 'all',
 '-',
 'top',
 'notch',
 'in',
 'every',
 'way',
 '!']

In [ ]:
# แปลงให้เป็น Token แล้วทำให้เป็นตัวเล็ก พร้อม ใส่ <sos> กับ <eos>
def tokenize_example(example, en_nlp, th_nlp, max_length, lower, sos_token, eos_token):
    en_tokens = [token.text for token in en_nlp.tokenizer(example["input_text"])][:max_length]
    th_tokens = th_nlp(example["translated_text"], engine="newmm")[:max_length]
    if lower:
        en_tokens = [token.lower() for token in en_tokens]
        # th_tokens = [token.lower() for token in th_tokens]
    en_tokens = [sos_token] + en_tokens + [eos_token]
    th_tokens = [sos_token] + th_tokens + [eos_token]
    return {"en_tokens" : en_tokens, "th_tokens" : th_tokens} # เพิ่ม list เข้าไป ซึ่งก็คือ en_tokens และ de_tokens

In [ ]:
max_length = 1000
lower = True
sos_token = "<sos>"
eos_token = "<eos>"

fn_kwargs = {
    "en_nlp" : en_nlp,
    "th_nlp" : th_nlp,
    "max_length" : max_length,
    "lower" : lower,
    "sos_token" : sos_token,
    "eos_token" : eos_token
}

train_data = train_data.map(tokenize_example, fn_kwargs=fn_kwargs)

In [ ]:
train_data[0]

{'input_text': 'Service, scallops, all - top notch in every way!',
 'translated_text': 'บริการหอยเชลล์ทั้งหมด - โดดเด่นในทุกด้าน!',
 'en_tokens': ['<sos>',
  'service',
  ',',
  'scallops',
  ',',
  'all',
  '-',
  'top',
  'notch',
  'in',
  'every',
  'way',
  '!',
  '<eos>'],
 'th_tokens': ['<sos>',
  'บริการ',
  'หอย',
  'เชลล์',
  'ทั้งหมด',
  ' ',
  '-',
  ' ',
  'โดดเด่น',
  'ใน',
  'ทุก',
  'ด้าน',
  '!',
  '<eos>']}

In [ ]:
# # แปลงให้เป็น Token แล้วทำให้เป็นตัวเล็ก พร้อม ใส่ <sos> กับ <eos>
# def thai_tokenize_example(sentence, th_nlp, max_length, sos_token, eos_token):
#     th_tokens = th_nlp(sentence, engine="newmm")[:max_length]
#     if lower:
#         pass
#         # th_tokens = [token.lower() for token in th_tokens]
#     th_tokens = [sos_token] + th_tokens + [eos_token]
#     return th_tokens # เพิ่ม list เข้าไป ซึ่งก็คือ en_tokens และ de_tokens

In [ ]:
# max_length = 1000
# sos_token = "<sos>"
# eos_token = "<eos>"

# fn_kwargs = {
#     "th_nlp" : th_nlp,
#     "max_length" : max_length,
#     "sos_token" : sos_token,
#     "eos_token" : eos_token
# }

# x = thai_tokenize_example("ฉันรักเธอ", th_nlp, max_length, sos_token, eos_token)

### Vocabularies

In [ ]:
# !pip install torch==2.2.0+cu118 torchvision -f https://download.pytorch.org/whl/torch_stable.html

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import random
import numpy as np
import spacy
import datasets
import torchtext
import tqdm
# import evaluate


In [ ]:
import tqdm
# import evaluate

min_freq = 2
unk_token = "<unk>"
pad_token = "<pad>"

special_tokens = [
    unk_token,
    pad_token,
    sos_token,
    eos_token
]

en_vocab = torchtext.vocab.build_vocab_from_iterator(
    train_data["en_tokens"],
    min_freq=min_freq,
    specials=special_tokens
)

th_vocab = torchtext.vocab.build_vocab_from_iterator(
    train_data["th_tokens"],
    min_freq=min_freq,
    specials=special_tokens
)

In [ ]:
en_vocab.get_itos()[:20]

['<unk>',
 '<pad>',
 '<sos>',
 '<eos>',
 '.',
 'the',
 ',',
 'to',
 'was',
 'and',
 'a',
 '!',
 'is',
 'i',
 'because',
 'for',
 'of',
 'but',
 'in',
 'it']

In [ ]:
en_vocab.get_itos()[9]

'and'

In [ ]:
th_vocab.get_itos()[:20]

['<unk>',
 '<pad>',
 '<sos>',
 '<eos>',
 ' ',
 'ที่',
 'และ',
 'ไม่',
 'ฉัน',
 'ของ',
 'มี',
 'ใน',
 'มาก',
 '!',
 'แต่',
 'เพราะ',
 'อาหาร',
 'ดี',
 'ได้',
 'เป็น']

In [ ]:
en_vocab.get_stoi()["the"]

5

In [ ]:
en_vocab["the"]

5

In [ ]:
len(en_vocab), len(th_vocab)

(17029, 13363)

In [ ]:
"the" in en_vocab

True

In [ ]:
"The" in en_vocab

False

In [ ]:
assert en_vocab[unk_token] == th_vocab[unk_token]
assert en_vocab[pad_token] == th_vocab[pad_token]

unk_index = en_vocab[unk_token]
pad_index = en_vocab[pad_token]



In [ ]:
en_vocab.set_default_index(unk_index)
th_vocab.set_default_index(unk_index)

In [ ]:
en_vocab["The"]

0

In [ ]:
en_vocab.get_itos()[0]

'<unk>'

In [ ]:
tokens = ["i", "love", "watching", "crime", "show"]

In [ ]:
en_vocab.lookup_indices(tokens)

[13, 52, 896, 2687, 388]

In [ ]:
en_vocab.lookup_tokens(en_vocab.lookup_indices(tokens))

['i', 'love', 'watching', 'crime', 'show']

In [ ]:
def numericalize_example(example, en_vocab, th_vocab):
    en_ids = en_vocab.lookup_indices(example["en_tokens"])
    th_ids = th_vocab.lookup_indices(example["th_tokens"])
    return {"en_ids": en_ids, "th_ids": th_ids}

In [ ]:
fn_kwargs = {"en_vocab": en_vocab, "th_vocab": th_vocab}

train_data = train_data.map(numericalize_example, fn_kwargs=fn_kwargs)

In [ ]:
train_data[0]

{'input_text': 'Service, scallops, all - top notch in every way!',
 'translated_text': 'บริการหอยเชลล์ทั้งหมด - โดดเด่นในทุกด้าน!',
 'en_tokens': ['<sos>',
  'service',
  ',',
  'scallops',
  ',',
  'all',
  '-',
  'top',
  'notch',
  'in',
  'every',
  'way',
  '!',
  '<eos>'],
 'th_tokens': ['<sos>',
  'บริการ',
  'หอย',
  'เชลล์',
  'ทั้งหมด',
  ' ',
  '-',
  ' ',
  'โดดเด่น',
  'ใน',
  'ทุก',
  'ด้าน',
  '!',
  '<eos>'],
 'en_ids': [2, 27, 6, 2518, 6, 65, 59, 453, 1368, 18, 199, 167, 11, 3],
 'th_ids': [2, 26, 1309, 2253, 171, 4, 143, 4, 624, 11, 313, 413, 13, 3]}

In [ ]:
en_vocab.lookup_tokens(train_data[0]["en_ids"])

In [ ]:
# สรุป คือ เราได้สร้าง dictionary แปลงเป็นตัวเลข(index) พร้อม mapไปกับdatasetแต่ละตัว

### เปลี่ยน index ให้เป็น Tensor

เราจะเปลี่ยน object ของ Train_data (ในที่นี้คือ en_ids และ de_ids) ไปเป็น Torch Tensor

In [ ]:
data_type = "torch"
format_columns = ["en_ids", "th_ids"]

train_data = train_data.with_format(type=data_type, columns=format_columns, output_all_columns=True)

In [ ]:
train_data[0]

In [ ]:
type(train_data[0]["en_ids"])

In [47]:
def get_collect_fn(pad_index):
    def collate_fn(batch):
        batch_en_ids = [example["en_ids"] for example in batch]
        batch_th_ids = [example["th_ids"] for example in batch]
        batch_en_ids = nn.utils.rnn.pad_sequence(batch_en_ids, padding_value=pad_index)
        batch_th_ids = nn.utils.rnn.pad_sequence(batch_th_ids, padding_value=pad_index)
        batch = {
            "en_ids" : batch_en_ids,
            "th_ids" : batch_th_ids
        }
        return batch

    return collate_fn

In [ ]:
def get_data_loader(dataset, batch_size, pad_index, shuffle=False):
    collate_fn = get_collect_fn(pad_index)
    data_loader = torch.utils.data.DataLoader(
        dataset=dataset,
        batch_size=batch_size,
        collate_fn=collate_fn,
        shuffle=shuffle
    )
    return data_loader

In [49]:
batch_size = 128
train_data_loader = get_data_loader(train_data, batch_size, pad_index, shuffle=True)

### Build the Model

In [50]:
class Encoder(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, n_layers, dropout):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout=dropout)
        self.dropout = nn.Dropout(dropout)

    def forward(self, src):
        # src = [src length, batch size]
        embedded = self.dropout(self.embedding(src))
        # embedded = [src length, batch size, embedding dim]
        outputs, (hidden, cell) = self.rnn(embedded)
        # outputs = [src length, batch size, hidden dim * n directions]
        # hidden = [n layers * n directions, batch size, hidden dim]
        # cell = [n layers * n directions, batch size, hidden dim]
        # outputs are always from the top hidden layer
        return hidden, cell

In [51]:
class Decoder(nn.Module):
    def __init__(self, output_dim, embedding_dim, hidden_dim, n_layers, dropout):
        super().__init__()
        self.output_dim = output_dim
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.embedding = nn.Embedding(output_dim, embedding_dim)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout=dropout)
        self.fc_out = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, input, hidden, cell):
        # input = [batch size]
        # hidden = [n layers * n directions, batch size, hidden dim]
        # cell = [n layers * n directions, batch size, hidden dim]
        # n directions in the decoder will both always be 1, therefore:
        # hidden = [n layers, batch size, hidden dim]
        # context = [n layers, batch size, hidden dim]
        input = input.unsqueeze(0)
        # input = [1, batch size]
        embedded = self.dropout(self.embedding(input))
        # embedded = [1, batch size, embedding dim]
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        # output = [seq length, batch size, hidden dim * n directions]
        # hidden = [n layers * n directions, batch size, hidden dim]
        # cell = [n layers * n directions, batch size, hidden dim]
        # seq length and n directions will always be 1 in this decoder, therefore:
        # output = [1, batch size, hidden dim]
        # hidden = [n layers, batch size, hidden dim]
        # cell = [n layers, batch size, hidden dim]
        prediction = self.fc_out(output.squeeze(0))
        # prediction = [batch size, output dim]
        return prediction, hidden, cell

In [52]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        assert (
            encoder.hidden_dim == decoder.hidden_dim
        ), "Hidden dimensions of encoder and decoder must be equal!"
        assert (
            encoder.n_layers == decoder.n_layers
        ), "Encoder and decoder must have equal number of layers!"

    def forward(self, src, trg, teacher_forcing_ratio):
        # src = [src length, batch size]
        # trg = [trg length, batch size]
        # teacher_forcing_ratio is probability to use teacher forcing
        # e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        batch_size = trg.shape[1]
        trg_length = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        # tensor to store decoder outputs
        outputs = torch.zeros(trg_length, batch_size, trg_vocab_size).to(self.device)
        # last hidden state of the encoder is used as the initial hidden state of the decoder
        hidden, cell = self.encoder(src)
        # hidden = [n layers * n directions, batch size, hidden dim]
        # cell = [n layers * n directions, batch size, hidden dim]
        # first input to the decoder is the <sos> tokens
        input = trg[0, :]
        # input = [batch size]
        for t in range(1, trg_length):
            # insert input token embedding, previous hidden and previous cell states
            # receive output tensor (predictions) and new hidden and cell states
            output, hidden, cell = self.decoder(input, hidden, cell)
            # output = [batch size, output dim]
            # hidden = [n layers, batch size, hidden dim]
            # cell = [n layers, batch size, hidden dim]
            # place predictions in a tensor holding predictions for each token
            outputs[t] = output
            # decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            # get the highest predicted token from our predictions
            top1 = output.argmax(1)
            # if teacher forcing, use actual next token as next input
            # if not, use predicted token
            input = trg[t] if teacher_force else top1
            # input = [batch size]
        return outputs

In [53]:
input_dim = len(th_vocab)
output_dim = len(en_vocab)
encoder_embedding_dim = 256
decoder_embedding_dim = 256
hidden_dim = 512
n_layers = 2
encoder_dropout = 0.5
decoder_dropout = 0.5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

encoder = Encoder(
    input_dim,
    encoder_embedding_dim,
    hidden_dim,
    n_layers,
    encoder_dropout,
)

decoder = Decoder(
    output_dim,
    decoder_embedding_dim,
    hidden_dim,
    n_layers,
    decoder_dropout,
)

model = Seq2Seq(encoder, decoder, device).to(device)

In [54]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)

model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(13363, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(17029, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (fc_out): Linear(in_features=512, out_features=17029, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [55]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"The model has {count_parameters(model):,} trainable parameters")

The model has 23,872,645 trainable parameters


In [56]:
optimizer = optim.Adam(model.parameters())

In [57]:
criterion = nn.CrossEntropyLoss(ignore_index=pad_index)

In [58]:
def train_fn(
    model, data_loader, optimizer, criterion, clip, teacher_forcing_ratio, device
):
    model.train()
    epoch_loss = 0
    for i, batch in enumerate(data_loader):
        src = batch["th_ids"].to(device)
        trg = batch["en_ids"].to(device)
        # src = [src length, batch size]
        # trg = [trg length, batch size]
        optimizer.zero_grad()
        output = model(src, trg, teacher_forcing_ratio)
        # output = [trg length, batch size, trg vocab size]
        output_dim = output.shape[-1]
        output = output[1:].view(-1, output_dim)
        # output = [(trg length - 1) * batch size, trg vocab size]
        trg = trg[1:].view(-1)
        # trg = [(trg length - 1) * batch size]
        loss = criterion(output, trg)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        epoch_loss += loss.item()
    return epoch_loss / len(data_loader)

In [59]:
def evaluate_fn(model, data_loader, criterion, device):
    model.eval()
    epoch_loss = 0
    with torch.no_grad():
        for i, batch in enumerate(data_loader):
            src = batch["th_ids"].to(device)
            trg = batch["en_ids"].to(device)
            # src = [src length, batch size]
            # trg = [trg length, batch size]
            output = model(src, trg, 0)  # turn off teacher forcing
            # output = [trg length, batch size, trg vocab size]
            output_dim = output.shape[-1]
            output = output[1:].view(-1, output_dim)
            # output = [(trg length - 1) * batch size, trg vocab size]
            trg = trg[1:].view(-1)
            # trg = [(trg length - 1) * batch size]
            loss = criterion(output, trg)
            epoch_loss += loss.item()
    return epoch_loss / len(data_loader)

In [60]:
n_epochs = 10
clip = 1.0
teacher_forcing_ratio = 0.5

for epoch in tqdm.tqdm(range(n_epochs)):
    train_loss = train_fn(
        model,
        train_data_loader,
        optimizer,
        criterion,
        clip,
        teacher_forcing_ratio,
        device,
    )
    # valid_loss = evaluate_fn(
    #     model,
    #     valid_data_loader,
    #     criterion,
    #     device,
    # )
    # if valid_loss < best_valid_loss:
    #     best_valid_loss = valid_loss
    #     torch.save(model.state_dict(), "Toilet_model.pt")
    print(f"\tTrain Loss: {train_loss:7.3f} | Train PPL: {np.exp(train_loss):7.3f}")
    # print(f"\tValid Loss: {valid_loss:7.3f} | Valid PPL: {np.exp(valid_loss):7.3f}")

 10%|█         | 1/10 [05:28<49:12, 328.03s/it]

	Train Loss:   6.098 | Train PPL: 445.032


 20%|██        | 2/10 [10:55<43:40, 327.59s/it]

	Train Loss:   5.468 | Train PPL: 236.905


 30%|███       | 3/10 [16:22<38:10, 327.23s/it]

...

	Train Loss:   4.258 | Train PPL:  70.685


 90%|█████████ | 9/10 [49:03<05:26, 326.63s/it]

	Train Loss:   4.133 | Train PPL:  62.365


100%|██████████| 10/10 [54:31<00:00, 327.11s/it]

	Train Loss:   3.998 | Train PPL:  54.480


In [69]:
def translate_sentence(
    sentence,
    model,
    en_nlp,
    th_nlp,
    en_vocab,
    th_vocab,
    lower,
    sos_token,
    eos_token,
    device,
    max_output_length=25,
):
    model.eval()
    with torch.no_grad():
        if isinstance(sentence, str):
            tokens = th_nlp(sentence, engine="newmm")
        else:
            tokens = [token for token in sentence]
        if lower:
            tokens = [token.lower() for token in tokens]
        tokens = [sos_token] + tokens + [eos_token]
        ids = th_vocab.lookup_indices(tokens)
        tensor = torch.LongTensor(ids).unsqueeze(-1).to(device)
        hidden, cell = model.encoder(tensor)
        inputs = en_vocab.lookup_indices([sos_token])
        for _ in range(max_output_length):
            inputs_tensor = torch.LongTensor([inputs[-1]]).to(device)
            output, hidden, cell = model.decoder(inputs_tensor, hidden, cell)
            predicted_token = output.argmax(-1).item()
            inputs.append(predicted_token)
            if predicted_token == en_vocab[eos_token]:
                break
        tokens = en_vocab.lookup_tokens(inputs)
    return tokens

In [329]:
sentence = "บริการแย่มาก ฉันจะให้คะแนนดาวศูนย์ถ้าเป็นไปได้ พวกเขาไม่ได้ให้บริการเราเพราะเรากำลังรอเพื่อนที่มาถึง การบริการลูกค้าที่แย่มาก อย่าไปที่นั่นหากคุณอยู่ในกลุ่ม"
expected_translation = "Bad service. I would rate zero star if possible. They didn't serve us because we were waiting for friends coming. Extremely bad service. Don't go there if you are in a group"

sentence2 = "เพื่อนของฉันโรงรถใหญ่กว่าบ้านทั้งหลังของเรา เพราะบ้านของเรามีชั้นเดียว"
expected_translation2 = "My friends garage was even bigger than our entire home, because the home is one story."

sentence3 = "เธอเติมน้ำผึ้งลงในชาแทนน้ำตาล เพราะน้ำผึ้งดีต่อสุขภาพสำหรับเธอมากกว่า"
expected_translation3 = "She sweetens het tea with honey instead of sugar, because the honey is healthier for her."

sentence4 = "อาหารไทยจืดชืด บรรยากาศดี"
expected_translation4 = "Food was bland. Ambiance was nice."

sentence5 = "วิกตอเรียให้ขวดนมสูตรสำหรับแพทริเซียทุกๆ สองชั่วโมงเพราะแพทริเซียเป็นลูกของเธอ"
expected_translation5 = "Victoria gets a bottle of formula for Patricia every couple of hours because Patricia is her child."

In [330]:
translation = translate_sentence(
    sentence,
    model,
    en_nlp,
    th_nlp,
    en_vocab,
    th_vocab,
    lower,
    sos_token,
    eos_token,
    device,
)
translation2 = translate_sentence(
    sentence2,
    model,
    en_nlp,
    th_nlp,
    en_vocab,
    th_vocab,
    lower,
    sos_token,
    eos_token,
    device,
)
translation3 = translate_sentence(
    sentence3,
    model,
    en_nlp,
    th_nlp,
    en_vocab,
    th_vocab,
    lower,
    sos_token,
    eos_token,
    device,
)
translation4 = translate_sentence(
    sentence4,
    model,
    en_nlp,
    th_nlp,
    en_vocab,
    th_vocab,
    lower,
    sos_token,
    eos_token,
    device,
)
translation5 = translate_sentence(
    sentence5,
    model,
    en_nlp,
    th_nlp,
    en_vocab,
    th_vocab,
    lower,
    sos_token,
    eos_token,
    device,
)

In [331]:
print(translation)
print(expected_translation)
print("\n")
print(translation2)
print(expected_translation2)
print("\n")
print(translation3)
print(expected_translation3)
print("\n")
print(translation4)
print(expected_translation4)
print("\n")
print(translation5)
print(expected_translation5)

['<sos>', 'service', 'was', 'horrible', '.', ' ', 'never', 'come', 'back', 'again', '.', ' ', 'we', 'never', 'come', 'back', 'again', '.', ' ', 'they', 'were', 'to', 'the', 'the', 'customer', 'service']
Bad service. I would rate zero star if possible. They didn't serve us because we were waiting for friends coming. Extremely bad service. Don't go there if you are in a group


['<sos>', 'my', 'sister', "'s", 'a', 'lot', 'better', 'than', 'the', 'my', 'house', 'because', 'the', 'house', 'was', 'very', '.', '<eos>']
My friends garage was even bigger than our entire home, because the home is one story.


['<sos>', 'she', 'bought', 'the', 'chocolate', 'instead', 'of', 'the', 'pie', ',', 'because', 'she', 'was', 'the', 'the', 'the', 'the', '.', '<eos>']
She sweetens het tea with honey instead of sugar, because the honey is healthier for her.


['<sos>', 'the', 'food', 'is', 'good', '.', '<eos>']
Food was bland. Ambiance was nice.


['<sos>', 'victoria', 'brought', 'a', 'to', 'for', 'patricia

In [273]:
translations = [
    translate_sentence(
        train_data["translated_text"],
        model,
        en_nlp,
        th_nlp,
        en_vocab,
        th_vocab,
        lower,
        sos_token,
        eos_token,
        device,
    )
    for example in tqdm.tqdm(train_data)
]





  0%|          | 0/59859 [00:00<?, ?it/s]



  0%|          | 1/59859 [00:02<47:18:55,  2.85s/it]

KeyboardInterrupt: 